In [14]:
# creating spectrograms from all the files, and saving split labelled versions to disk ready for machine learning
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
import cPickle as pickle
import numpy as np

# from data_helpers import load_annotations

annotation_pkl_dir = '/media/michael/Seagate/engage/alison_data/golden_set/extracted/annotations/'
savedir = '/media/michael/Seagate/engage/alison_data/golden_set/extracted/specs/'
base_path = '/media/michael/Seagate/engage/alison_data/golden_set/labels/Golden/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def gen_spec(x, sr):
    specNStepMod = 0.01    # horizontal resolution of spectogram 0.01
    specNWinMod = 0.03     # vertical resolution of spectogram 0.03

    ## Parameters
    nstep = int(sr * specNStepMod)
    nwin  = int(sr * specNWinMod)
    nfft = nwin

    # Get all windows of x with length n as a single array, using strides to avoid data duplication
    #shape = (nfft, len(range(nfft, len(x), nstep)))
    shape = (nfft, ((x.shape[0] - nfft - 1)/nstep)+1)
    strides = (x.itemsize, nstep*x.itemsize)
    x_wins = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)

    # Apply hamming window
    x_wins_ham = np.hamming(x_wins.shape[0])[..., np.newaxis] * x_wins

    # compute fft
    fft_mat = np.fft.fft(x_wins_ham, n=nfft, axis=0)[:(nfft/2), :]

    # log magnitude
    fft_mat_lm = np.log(np.abs(fft_mat))
    fft_mat = np.abs(fft_mat)

    return fft_mat

In [20]:
# create (multi-channel?) spectrogram
files = os.listdir(annotation_pkl_dir)

for fname in files:
    if fname.endswith('*.pkl'):

        with open(annotation_pkl_dir + fname) as f:
            annots, wav, sample_rate = pickle.load(f)

        spec = gen_spec(wav, sample_rate)

        with open(savedir + fname, 'w') as f:
            pickle.dump(spec, f, -1)
            
        print fname